In [1]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [ ]:
###########    FUNCIONES   ##########
#Se calculan los outliers para identificación de posibles datos atipicos
#Q1–1.5·RIC o superiores a Q3+1.5·RIC.
def outliers(variable):
  LI=quantile(variable,0.25)-(1.5*(quantile(variable,0.75)-quantile(variable,0.25)))
  LS=quantile(variable,0.75)+(1.5*(quantile(variable,0.75)-quantile(variable,0.25)))
  return(c(LI,LS))



#Creamos variables temporales donde excluimos los registros que están por fuera de los outliers
categorizacion <- function(variable,outliers,reciencia){
  LI=outliers[1]
  LS=outliers[2]
  VarTemp=variable[variable>=LI & variable<=LS]
  
  # break_1: quantiles sin outliers
  breaks_1=c(min(variable)-0.001,quantile(VarTemp,probs = seq(0,1,0.2))[2:5],max(variable)+0.001)
  
  # break_2: quantiles con outliers
  breaks_2=quantile(variable,probs = seq(0,1,0.2))
  breaks_2[1]=breaks_2[1]-0.001
  breaks_2[6]=breaks_2[6]+0.001
  
  # break_3: rangos fijos con outliers
  breaks_3=seq(min(variable)-0.001,max(variable)+0.001,length.out= 6)
  
  # Tres casos para asignar puntajes
  if(reciencia==FALSE){
    # Usar el break sin outliers sin que quantiles repitan valores
    if(sum(duplicated(breaks_1))==0){
      categorias=cut(variable, breaks=breaks_1,labels = 1:5) %>% as.numeric()
    }
    # Usar el break con outliers sin que quantiles repitan valores
    if(sum(duplicated(breaks_1))!=0 & sum(duplicated(breaks_2))==0){
      categorias=cut(variable, breaks=breaks_2,labels = 1:5) %>% as.numeric()
    }
    # Usar el break con outliers sin usar quantiles sino rangos fijos
    if(sum(duplicated(breaks_1))!=0 & sum(duplicated(breaks_2))!=0){
      categorias=cut(variable, breaks=breaks_3,labels = 1:5) %>% as.numeric()
    }
  }
  # restar 6 y multiplicar -1 invierte el puntaje para Recencia, es decir, si el puntaje mayor es 5 para los valores mas grandes 
  # en los otros casos, en Recencia los menores son los de mayor puntaje
  if(reciencia==TRUE){
    if(sum(duplicated(breaks_1))==0){
      categorias=((cut(variable, breaks=breaks_1,labels = 1:5) %>% as.numeric())-6)*(-1)
    }
    if(sum(duplicated(breaks_1))!=0 & sum(duplicated(breaks_2))==0){
      categorias=((cut(variable, breaks=breaks_2,labels = 1:5) %>% as.numeric())-6)*(-1)
    }
    if(sum(duplicated(breaks_1))!=0 & sum(duplicated(breaks_2))!=0){
      categorias=((cut(variable, breaks=breaks_3,labels = 1:5) %>% as.numeric())-6)*(-1)
    }
  }
  
  return(categorias)
}



RFMManar <- function(BD){
  rfm_df=BD
  LRecencia=outliers(rfm_df$Recencia)
  LMonto=outliers(rfm_df$Monto)
  LFrecuencia=outliers(rfm_df$Frecuencia)
  
  rfm_df$ScoreRecencia=categorizacion(rfm_df$Recencia,LRecencia,reciencia=TRUE)
  rfm_df$ScoreFrecuencia=categorizacion(rfm_df$Frecuencia,LFrecuencia,reciencia=FALSE)
  rfm_df$ScoreMonto=categorizacion(rfm_df$Monto,LMonto,reciencia=FALSE)
  
  rfm_df$Score=NULL
  rfm_df$Score=as.numeric(paste(rfm_df$ScoreRecencia,rfm_df$ScoreFrecuencia,rfm_df$ScoreMonto,sep=""))
  
  return(rfm_df)
}

In [2]:
# Carga de la base
inpath = "C:/Users/Diego Torres/OneDrive - Manar Technologies SAS/Carvajal/Bases/"

BD_ini = pd.read_excel(inpath + 'BD_Carvajal1.xlsx',
                       sheet_name="Hoja2")

In [3]:
#################
# Descriptivos
print(BD_ini.sample(5))
print(' ')
print('Summary:')
BD_ini.info()
print(' ')
print('Descriptivos:')
print(BD_ini.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(BD_ini.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(BD_ini.shape)

       codigo cliente  nombre cliente  codigo producto   
10032     13330695284     13330695284         32408002  \
42898     13330000912     13330000912         32105003   
47622       113202668       113202668         31000686   
58086       113299696       113299696         31000868   
66616       112388082       112388082         31705007   

                                         nombre producto fecha venta   
10032              DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-03-23  \
42898        MT-IBC MENSUALIDAD ADMINISTRACION CENT PLUS  2022-06-02   
47622  **EXC EB - MENSUALIDAD VARIABLE FACTURA ELECTR...  2022-06-14   
58086                            **EXC EB - PYME PLAN 10  2022-07-05   
66616  DIF MANTENIMIENTO A DISTANCIA DE PROGRAMAS Y E...  2022-07-25   

       cantidad facturado  valor facturado                 Tipo  
10032                   1          45411.0       CEN FINANCIERO  
42898                   1         402403.0    CEN TRANSACCIONAL  
47622               